In [16]:
# general
import pandas as pd
import geopandas

# SQL 
import sqlite3
from sqlalchemy import create_engine

# utility
from tqdm import tqdm

In [17]:
geo = geopandas.read_file("data/geo/Municipios_Colombia.geojson")
AGROSAVIA_df = pd.read_csv("data/suelos_geolocalizado.csv")

# Preparación de los datos

## Tabla `municipios`

In [18]:
municipios = geo.set_index("id")[["ID_DEP","departamento","municipio","LATITUD","LONGITUD"]]

municipios["altitud"] = pd.read_csv(
    "data/geo/altitud.csv",
    dtype={"id":str,"altitud":float}
).set_index("id").altitud.round(2)

municipios.index.rename("cod_municipio",inplace=True)
municipios.rename(columns={
    "ID_DEP":"cod_departamento","ID_MUN":"cod_municipio",
    "LATITUD":"latitud","LONGITUD":"longitud"
},inplace=True)

In [19]:
municipios.sample(5)

,cod_departamento,departamento,municipio,latitud,longitud,altitud
cod_municipio,,,,,,
70124,70,sucre,caimito,8.792312,-75.131911,27.32
54800,54,norte de santander,teorama,8.760014,-73.157141,735.35
05044,05,antioquia,anzá,6.308040,-75.912591,1477.83
94885,94,guainía,la guadalupe,1.404540,-67.001518,102.94
13688,13,bolívar,santa rosa del sur,7.772519,-74.262728,696.64


## Tablas `zonas` y  `departamentos`

In [20]:
zonas = pd.DataFrame({
    "AMA":"Amazonía",
    "AND":"Andina",
    "ORI":"Orinoquía",
    "PAC":"Pacífico",
    "CAR":"Caribe"
}.items(),columns=["cod_region","region"]).set_index("cod_region")
zonas

,region
cod_region,
AMA,Amazonía
AND,Andina
ORI,Orinoquía
PAC,Pacífico
CAR,Caribe


In [21]:
zonas_dict = {
    "AMA":["18","50","86","91","94","95","97","99"],
    "AND":["05","11","15","17","18","19","20","25","27","41","52","54","63","66","68","73","76","86"],
    "ORI":["50","81","85","99"],
    "PAC":["19","27","52","76"],
    "CAR":["08","13","20","23","44","47","70","88"]
}

In [22]:
departamentos = municipios[["cod_departamento","departamento"]].drop_duplicates().set_index("cod_departamento")
departamentos = pd.concat([
    departamentos,
    pd.DataFrame(index=departamentos.index,columns=zonas_dict.keys())
],axis=1)
for col,deps in zonas_dict.items():
    departamentos.loc[deps,col] = 1
departamentos.fillna(0,inplace=True)

In [23]:
# check that all departments have regions
(departamentos[zonas_dict.keys()].sum(axis=1) == 0).sum()

0

In [24]:
departamentos.sample(10)

,departamento,AMA,AND,ORI,PAC,CAR
cod_departamento,,,,,,
66,risaralda,0,1,0,0,0
54,norte de santander,0,1,0,0,0
18,caquetá,1,1,0,0,0
20,cesar,0,1,0,0,1
08,atlántico,0,0,0,0,1
88,"archipiélago de san andrés, providencia y sant...",0,0,0,0,1
95,guaviare,1,0,0,0,0
44,la guajira,0,0,0,0,1
41,huila,0,1,0,0,0


## Tabla `analisis`

In [25]:
analisis = AGROSAVIA_df.drop(["departamento","municipio","id"],axis=1)
analisis.index.rename("id",inplace=True)
analisis.rename(columns={"geo_id":"cod_municipio"},inplace=True)

In [26]:
analisis.sample(5)

,cod_municipio,cultivo,fertilizantes,ph,materia_organica,fosforo,azufre,acidez,aluminio,calcio,...,drenaje_regular,riego_aspersión,riego_cañón,riego_goteo,riego_gravedad,riego_manguera,riego_microaspersión,riego_no_cuenta_con_riego,riego_no_indica,riego_por_inundación
id,,,,,,,,,,,,,,,,,,,,,
29456,76520,aguacate,15-15-15,6.31,5.460581,72.271222,6.416803,NaN,NaN,11.306880,...,0,1,0,0,0,0,0,0,0,0
14776,76520,pastos-estrella,no indica,6.80,3.903890,7.189390,60.352850,NaN,NaN,12.676531,...,0,1,0,0,0,0,0,0,0,0
15261,54001,papaya,no ha aplicado fertilizantes,6.30,0.448079,1.935000,6.907197,NaN,NaN,2.412630,...,0,0,0,0,0,0,0,1,0,0
40989,25662,café,no indica,4.98,2.704430,17.599015,4.299478,0.328038,0.260845,1.709760,...,0,0,0,0,0,0,0,1,0,0
18988,76828,mora,no indica,4.43,10.584798,1.935000,6.837829,5.156153,4.177272,0.852339,...,0,0,0,0,0,0,0,1,0,0


In [27]:
analisis.columns

Index(['cod_municipio', 'cultivo', 'fertilizantes', 'ph', 'materia_organica',
       'fosforo', 'azufre', 'acidez', 'aluminio', 'calcio', 'magnesio',
       'potasio', 'sodio', 'cice', 'ce', 'hierro_olsen', 'cobre', 'manganeso',
       'zinc_olsen', 'boro', 'hierro_doble_acido', 'cobre_doble_acido',
       'manganeso_doble_acido', 'zinc_doble_acido', 'estado_establecido',
       'estado_no_indica', 'estado_por_establecer',
       'tiempo_establecimiento_de_0_a_1_año',
       'tiempo_establecimiento_de_1_a_5_años',
       'tiempo_establecimiento_de_5_a_10_años',
       'tiempo_establecimiento_mas_de_10_años',
       'tiempo_establecimiento_no_aplica', 'tiempo_establecimiento_no_indica',
       'topografia_ligeramente_ondulado', 'topografia_moderadamente_ondulado',
       'topografia_no_indica', 'topografia_ondulado', 'topografia_pendiente',
       'topografia_pendiente_fuerte', 'topografia_pendiente_leve',
       'topografia_pendiente_moderada', 'topografia_plano', 'drenaje_bueno',
    

## Tabla `variables`

In [28]:
variables = pd.DataFrame({
    "acidez":"Acidez",
    "aluminio":"Aluminio",
    "azufre":"Azufre",
    "boro":"Boro",
    "calcio":"Calcio",
    "ce":"Coductividad eléctrica",
    "cice":"Coef. intercambio catiónico",
    "cobre":"Cobre",
    "cobre_doble_acido":"Cobre doble ácido",
    "fosforo":"Fósforo",
    "hierro_doble_acido":"Hierro doble ácido",
    "hierro_olsen":"Hierro Olsen",
    "magnesio":"Magnesio",
    "manganeso":"Manganeso",
    "manganeso_doble_acido":"Manganeso doble ácido",
    "materia_organica":"Materia orgánica",
    "ph":"Ph",
    "potasio":"Potasio",
    "sodio":"Sodio",
    "zinc_olsen":"Zinc Olsen",
}.items(),columns=["codigo","variable"]).set_index("codigo")

variables.sample(5)

,variable
codigo,
azufre,Azufre
cobre,Cobre
materia_organica,Materia orgánica
calcio,Calcio
magnesio,Magnesio


# Creación de la base de datos

In [29]:
tablas = ["municipios","departamentos","zonas","analisis","variables"]

In [30]:
engine = create_engine("sqlite:///data/database.db")

for name in tablas:
    tabla = locals()[name]
    con = engine.connect()
    
    tabla.to_sql(name, con, if_exists='replace')
    
    con.close()
    print(f"table `{name}` written")

table `municipios` written
table `departamentos` written
table `zonas` written
table `analisis` written
table `variables` written
